In [1]:
import pandas as pd
import numpy as np

Naive Bayes classification model is based on bayes theoram of conditional probability.

The formula for which is: 

$p(A|B) = \frac{p(B|A).p(A)}{p(B)}$

The above is read as the **_probability of event A occuring given that event B has alreay occured_**


In [2]:
data = pd.read_csv('heart-disease-data/heart.csv')

In [3]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


For our algorithm to work we only need discrete varaibles so we will drop any continous varaibles

In [4]:
data.drop(["age", "trestbps", "chol", "thalach", "oldpeak", "slope"],axis = 1 ,inplace=True)

In [5]:
data.head()

,sex,cp,fbs,restecg,exang,ca,thal,target
0,1,3,1,0,0,0,1,1
1,1,2,0,1,0,0,2,1
2,0,1,0,0,0,0,2,1
3,1,1,0,1,0,0,2,1
4,0,0,0,1,1,0,2,1


Lets build a helper function to calculate the conditional probabilities of individual inputs

In [6]:
#Calcualting probabilites for inputs independantly

def get_probabilities_for_inputs(n, column_name, data_frame):
    
    temp = data_frame[column_name] #isolate targetted column
    temp = temp.value_counts() #get counts of occurences of each input variable
    
    return (temp/n) #return probiblity of occurence by dividing with total no. of data points
    

In [20]:
#calculating conditional probability

def get_conditional_probabilities(data_frame, n,target, given):
    
    focused_data = data[[target, given]] #isolate target column an dfocus input column

    targets_unique = data[target].unique()
    inputs_unique = data[given].unique()
    
    groups = focused_data.groupby(by = [given, target]).size().reset_index()
    groups[0] = groups[0]/ n
    
    print("Individual probabilities: ",groups)
    for targets in targets_unique:
        current_target_length = len(focused_data[focused_data[target] == targets])
        print("Working for target: ", targets)
        print("length of filtered df:", current_target_length)
        groups[0] = np.where(groups[target] == targets, groups[0].div(current_target_length),groups[0])
    print("After conditional:", groups)  
    
    
    
            
            

In [8]:
n = len(data)

In [9]:
f = lambda lst: get_probabilities_for_inputs(n, lst, data)
probablities = list(map(f,data.keys()))

In [10]:
probablities

[1    0.683168
 0    0.316832
 Name: sex, dtype: float64,
 0    0.471947
 2    0.287129
 1    0.165017
 3    0.075908
 Name: cp, dtype: float64,
 0    0.851485
 1    0.148515
 Name: fbs, dtype: float64,
 1    0.501650
 0    0.485149
 2    0.013201
 Name: restecg, dtype: float64,
 0    0.673267
 1    0.326733
 Name: exang, dtype: float64,
 0    0.577558
 1    0.214521
 2    0.125413
 3    0.066007
 4    0.016502
 Name: ca, dtype: float64,
 2    0.547855
 3    0.386139
 1    0.059406
 0    0.006601
 Name: thal, dtype: float64,
 1    0.544554
 0    0.455446
 Name: target, dtype: float64]

In [21]:
get_conditional_probabilities(data, n, "target", "sex")

Individual probabilities:     sex  target         0
0    0       0  0.079208
1    0       1  0.237624
2    1       0  0.376238
3    1       1  0.306931
Working for target:  1
length of filtered df: 165
Working for target:  0
length of filtered df: 138
After conditional:    sex  target         0
0    0       0  0.000574
1    0       1  0.001440
2    1       0  0.002726
3    1       1  0.001860
